In [1]:
pwd

'/home/rnshishir/deepmed/CPRD_Cut22/debug'

In [2]:
import pyspark

In [3]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge, cohort_select
from CPRD.functions import merge
from utils.utils import save_obj

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [5]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.txt',
  'op_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_09

In [7]:
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.config.utils import *
import CPRD.base.table as cprd_table
import pandas as pd
from CPRD.functions.modalities import *

from CPRD.functions.Causal import *
from pyspark.sql import Window
from CPRD.functions.MedicalDictionary import *
from CPRD.config.utils import *
from CPRD.config.spark import read_parquet, read_txtzip, read_txt
from CPRD.functions.cohort_select_causal import *
from CPRD.functions.predictor_extractor import *

In [8]:
dict4recs = load_obj( '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/Documentation/Set 21/linkage_coverage_dictv')

dict4recs


{'hes_apc': ['01/04/1997', '31/10/2020'],
 'hes_acp': ['01/10/1997', '31/03/2008'],
 'hes_ccare': ['01/04/2008', '31/10/2020'],
 'hes_op': ['01/04/2003', '31/10/2020'],
 'hes_ae': ['01/04/2007', '31/03/2020'],
 'hes_did': ['01/04/2012', '31/10/2020'],
 'ons_death': ['02/01/1998', '16/11/2020'],
 'ncras_cr': ['01/01/1990', '31/12/2018'],
 'ncras_sact': ['01/01/2014', '31/12/2018'],
 'ncras_rtds': ['01/04/2012', '31/12/2018']}

# raw data tables

In [9]:
# patients table 
pat = tables.retrieve_patient(file['patient'], spark)
# pat.show()

In [ ]:
# practice table 
prac = tables.retrieve_practice(file['practice'], spark)
# prac.show()

In [ ]:
# consultation table
cons = tables.retrieve_consultation(file['consultation'], spark)
# cons.show()

In [ ]:
# raw clinical table - has a bunch of things including measurements and the diagnoses of course
clin = tables.retrieve_clinical(file['clinical'], spark)
# clin.show()

In [ ]:
# raw medications file
meds =  tables.retrieve_therapy(file['therapy'], spark)
# meds.show()

In [ ]:
#  death table which is ONS linked
dth = tables.retrieve_death(file['death'], spark)


In [ ]:
# hes APC - admitted patient care
hesapc = tables.retrieve_hes_diagnoses(file['diagnosis_hes'], spark)

In [ ]:
# hes procedures from , i believe, APC - admitted patient care
hesproc = tables.retrieve_hes_proc(file['proc_hes'], spark)

# example retrieval of demographics

In [11]:
patient = tables.retrieve_patient(dir=file['patient'], spark=spark)
practice = tables.retrieve_practice(dir=file['practice'], spark=spark)
# we wish to see people eligible for anlayses (i.e., clean of recording errors/etc) that linked to practice
demographics = tables.retrieve_demographics(patient=patient, practice=practice, practiceLink=True)
demographics.count()

34283066

In [12]:
# 34283066 patients - roughly 34 mil

# example retrieval of diag (please don't save any more copies anywhere!

In [ ]:
diag = retrieve_diagnoses(file, spark, mapping='sno2icd', duration=(1985, 2021), practiceLink=True)
# diag.write.parquet('/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
# can read a copy of the diag file with GPlinked patients with HESAPC records saved here ^^^

# total rows available 947995617
# patients with records with med->sno-icd 29724610
# patients with records with med->read-icd 28674531


# example retrieval of bp (please don't save any more copies anywhere!

In [ ]:
bp = retrieve_systolic_bp_measurement(file, spark, duration=(1985, 2021), usable_range=(50, 300))
# bp.write.parquet('/home/shared/shishir/AurumOut/rawDat/AllBP_praclinkage_1985_2021.parquet')
# can read a copy of the BP file with GPlinked patients here ^^^^


# example retrieval of medications 
FOR THIS DO NOT SAVE ANY COPIES ANYWHERE AS IT IS TOO BIG A FILE AND WASTES SPACE!!!

In [ ]:
meds = retrieve_medications(file, spark, mapping='bnfvtm', duration=(1985, 2021), demographics=None, practiceLink=True)
# bnf and vtm mapping to a homogeneous coding language

    
    

In [ ]:
# if you wish to look at the raw mapping tables here it is:
proddict = read_txt(spark.sc, spark.sqlContext, path='/home/workspace/datasets/cprd/cprd2021/202102_lookups/202102_EMISProductDictionary.txt')
vtmatc = read_txt(spark.sc, spark.sqlContext, path  ='/home/shared/zhengxian/maps/dmd_maps.txt').withColumnRenamed('snomed', 'dmdid')[['dmdid','atc','vtm']]


In [31]:
# counts of bnf and vtm in the medication dataset:
counts = read_parquet( spark.sqlContext , '/home/shared/shishir/AurumOut/Dicts/meds_medcount_full.parquet')

In [36]:
# counts.show(20)

# example medical dictionary for use

In [13]:
from CPRD.functions.MedicalDictionary import * 

md = MedicalDictionaryRiskPrediction(file, spark)

In [16]:
# all the phenotyping libraries that have been put together for the medical dictionary
md.componentDict.keys()

dict_keys(['dare2think_Pheno', 'dare2think_Meds', 'dare2think_Obs', 'dare2think_Proc', 'NC_lancet2022_Pheno', 'NC_lancet2022_Obs', 'DM_curated_Extra', 'Caliber2020_ICD10_Pheno'])

In [29]:
# query for af and stroke for example
out_af_stroke = md.queryDisease(['atrial fibrillation', 'stroke'])
out_af_stroke['stroke'].keys()

dict_keys(['medcode', 'ICD10', 'prodcode', 'OPCS'])

In [28]:
# perhaps you don't know how to exactly label the term so you do a find:
ischaem_quer = md.findItem('ischaem')
# now you get codes to phenoytpe all the actual keys below
allischaem = md.queryDisease(ischaem_quer)
allischaem.keys()

dict_keys(['ischaemic stroke', 'ischaemic heart disease', 'transient ischaemic attack'])